In [1]:
import os
import sqlite3 as sql
from datetime import datetime

import pandas as pd

# Coleta de dados do site do IBGE

In [2]:
# Coletando dados do IBGE
url = 'https://www.ibge.gov.br/explica/codigos-dos-municipios.php#RJ'

dadosIBGE = pd.read_html(url, match="Municípios do Rio de Janeiro")[0]

# Renomear colunas do df
dadosIBGE = dadosIBGE.rename(columns={'Municípios do Rio de Janeiro':'nmMunic','Códigos':'codMunic'})

# Alterando o nome do index
dadosIBGE.index.name = 'idMunic'

# Alterando o index para comelar em 1
dadosIBGE.index = dadosIBGE.index + 1

# Adicionoar uma coluna de data e hora decarga 
dtCarga = datetime.today().strftime('%d/%m/%Y %H:%M')
dadosIBGE['dtCarga'] = dtCarga

# dadosIBGE

# Criando o Banco de Dados ODS

In [3]:
# Manipulando o sistema de arquivos
path = os.getcwd()

BDODS = os.path.join(path, "pascoaODS.db")
BDDW = os.path.join(path, "pascoaDW.db")

# Manipulando os Bancos de Dados Criados

In [4]:
# Conectar no BDODS
conexaoODS = sql.connect(BDODS)

# Criar a tabela tbLogMunic e carregar os dados do DF dadosIBGE
dadosIBGE.to_sql('tbLogMunic', conexaoODS, if_exists="append")

# Confirmar a transação
conexaoODS.commit()

# Fechat a conexão
conexaoODS.close()

print("Cargo do BDODS concluída!")

Cargo do BDODS concluída!


In [5]:
# Conectar no BDDW
conexaoDW = sql.connect(BDDW)

# Selecionar somente as colunas para criação da dMunicipio
dadosIBGE = dadosIBGE[['nmMunic', 'codMunic']]

# Criar a tabela dMunicipio e carregar os dados do DF dadosIBGE
dadosIBGE.to_sql('dMunicipio', conexaoDW, if_exists="replace")

# Confirmar a transação
conexaoDW.commit()

# Fechat a conexão
conexaoDW.close()

print("Cargo do BDDW concluída!")

Cargo do BDDW concluída!


# Carga dos Dados de Cadastro das Delegacias de Polícia

In [6]:
# Definir o endereço fisico do arquivo CSV
pathDP  = os.path.join(path, 'DP.csv')

# Cria DataFrame com os dados das DPs
tbLogDP = pd.read_csv(pathDP)

# Criando coluna com a data da carga
tbLogDP['dtCarga'] = dtCarga
# Selecionado a coluna COD_DP como novo index
# tbLogDP = tbLogDP.set_index("COD_DP")

# Carga dos dados de cadastro dos Responsaveis pelas delegacia de policia

In [7]:
# Criar DataFrame com os dados com os reponsaveis pelas DPs
path_responsaveis = os.path.join(path, 'ResponsavelDP.csv')

tbLogRespDP = pd.read_csv(path_responsaveis)

# Criando coluna com a data da carga
tbLogRespDP['dtCarga'] = dtCarga

# Selecionado a coluna COD_DP como novo index
# tbLogRespDP = tbLogRespDP.set_index("COD_DP")

# Conectando no ODS

In [8]:
# Conectando no ODS
conexaoODS = sql.connect(BDODS)

# Definição de manipulação de dados em SQL
sql_ODS = conexaoODS.cursor()


# Inserindo Dados das Delegacias de Polícia, no SQLite

In [9]:
# Inserindo registros na tabela tbLogDP
print("Iniciando a inserção de dados na tabela tbLogDP.")
print("------------------------------------------------")

sql_ODS.executemany(''' INSERT INTO tbLogDP (codDP, nmDP, endereco, dtCarga) VALUES (?,?,?,?)''',tbLogDP.values.tolist()) 

# Confirmar a transação
conexaoODS.commit()

# Exibir uma mensagem de conclusão de carga
print(f"Carga Finalizada! {len(tbLogDP)} registros inseridos na tbLogDP")

Iniciando a inserção de dados na tabela tbLogDP.
------------------------------------------------
Carga Finalizada! 137 registros inseridos na tbLogDP


# Inserindo Dados dos Responsaveis pela Delegacias de Polícia, no SQLite


In [10]:
# Inserindo registros na tabela tbLogRespDP
print("Iniciando a inserção de dados na tabela tbLogRespDP.")
print("------------------------------------------------")

sql_ODS.executemany(''' INSERT INTO tbLogRespDP (codDP, nmResponsavel, dtCarga) VALUES (?,?,?)''',tbLogRespDP.values.tolist()) 

# Confirmar a transação
conexaoODS.commit()

# Exibir uma mensagem de conclusão de carga
print(f"Carga Finalizada! {len(tbLogRespDP)} registros inseridos na tbLogRespDP")

Iniciando a inserção de dados na tabela tbLogRespDP.
------------------------------------------------
Carga Finalizada! 137 registros inseridos na tbLogRespDP


# Validando Carga dos Dados de DP e de Responsável do DP

In [11]:
# Validadar a carga através de um select na tbLogDP
# pd.read_sql("SELECT * FROM tbLogDP", conexaoODS)

In [12]:
# Validadar a carga através de um select na tbLogRespDP
# pd.read_sql("SELECT * FROM tbLogRespDP", conexaoODS)

# Criando o DataFrame com o select de criação da dimensão DP (dDP), no DW

In [13]:
# definir a query de criação da dimensão dDp

qry_dDP = '''
    SELECT
        codDP,
        nmDP,
        endereco,
        nmResponsavel
    FROM
    (
        SELECT
            a.codDP,
            a.nmDP,
            a.endereco,
            b.nmResponsavel,
            MAX(a.dtCarga)
        FROM  tbLogDP a
        JOIN tbLogRespDP b ON (a.codDP  = b.codDP)
        GROUP BY 
            a.codDP,
            a.nmDP,
            a.endereco,
            b.nmResponsavel
    ) a
'''

dDP = pd.read_sql(qry_dDP, conexaoODS)

# Coletando dados dos Batalhões de Polícia Militar (BPM)

In [14]:
# Criar DataFrame com os dados dos BPMs
path_BPM = os.path.join(path, 'BPM.csv')
tbLogBPM = pd.read_csv(path_BPM)

# Criando coluna com a data da carga
tbLogBPM['dtCarga'] = dtCarga

# Construindo a Tebela de Log de Registros do BPM (tbLogBPM) no ODS

In [16]:
# Definir a query decriação da tabela
qry_tbLogBPM = """
    CREATE TABLE IF NOT EXISTS tbLogBPM(
    
    codBPM INTEGER,
    nmBPM VARCHAR (7),
    endereco VARCHAR (200),
    dtCarga DATETIME
)
"""

# executar a criação da tbLogBPM
sql_ODS.execute(qry_tbLogBPM)

# definição da qry de criação de index
tbLogBPM_codBPM_IDX = "CREATE INDEX IF NOT EXISTS tbLogBPM_codBPM_IDX ON tbLogBPM (codBPM)"

# executar a criação do index
sql_ODS.execute(tbLogBPM_codBPM_IDX)


# Inserindo Dados de BPM na Tabela tbLogBPM

In [18]:
# Inserindo registros na tabela tbLogBPM
print("Iniciando a inserção de dados na tabela tbLogBPM.")
print("------------------------------------------------")

sql_ODS.executemany(''' INSERT INTO tbLogBPM (codBPM, nmBPM, endereco, dtCarga) VALUES (?,?,?,?)''',tbLogBPM.values.tolist()) 

# Confirmar a transação
conexaoODS.commit()

# Exibir uma mensagem de conclusão de carga
print(f"Carga Finalizada! {len(tbLogBPM)} registros inseridos na tbLogBPM")

Iniciando a inserção de dados na tabela tbLogBPM.
------------------------------------------------
Carga Finalizada! 39 registros inseridos na tbLogBPM


# Fecando a conexão com o ODS

In [ ]:
# Confirmar a transação 
conexaoODS.commit()

# Fechar a conexão
conexaoODS.close()

# Conectando no DW e Informando ao python que usarei Linguagem SQL

In [37]:
# Conectar no BDDW
conexaoDW = sql.connect(BDDW)

# Definição de manipulação de dados em SQL
sql_DW = conexaoDW.cursor()

# Deletando os Dados Antigos e Reiniciando o Autoincremental da Chave

In [38]:
# Deletar os dados atuais
sql_DW.execute("DELETE FROM dDP") # A alegria de todo desenvolvedor

# Reiniciando o autoincremental da Chave
sql_DW.execute("UPDATE sqlite_sequence SET seq=0 WHERE name='dDP'")

# Confirmar a transação
conexaoDW.commit()


# Inserido dados na Dimensão DP (dDP), no DW

In [39]:
# Inserindo registros na tabela dDP
print("Iniciando a inserção de dados na tabela dDP.")
print("------------------------------------------------")

sql_DW.executemany(''' INSERT INTO dDP (codDP, nmDP, endereco, nmResponsavel) VALUES (?,?,?,?)''',dDP.values.tolist()) 

# Confirmar a transação
conexaoDW.commit()

# Exibir uma mensagem de conclusão de carga
print(f"Carga Finalizada! {len(dDP)} registros inseridos na dDP")

Iniciando a inserção de dados na tabela dDP.
------------------------------------------------
Carga Finalizada! 137 registros inseridos na dDP


# Validando Carga  dos Dados da Dimensão DP (dDP)

In [40]:
# Validadar a carga através de um select na dDP
pd.read_sql("SELECT * FROM dDP", conexaoDW)

,idDP,codDP,nmDP,endereco,nmResponsavel
0,1,1,001ª DP - Praça Mauá,Atendendo provisoriamente na sede da 4ª DP - P...,Delegado de Polícia José Luiz Silva Duarte
1,2,4,004ª DP - Praça da República,"Av. Presidente Vargas, 1100 - Centro, Rio de J...",Delegada de Polícia Patricia de Paiva Aguiar
2,3,5,005ª DP - Mem de Sá,"Avenida Gomes Freire, 320 - Centro, Rio de Jan...",Delegado de Polícia Bruno Gilaberte Freitas
3,4,6,006ª DP - Cidade Nova,"Rua Professor Clementino Fraga, 77 - Centro, R...",Delegado de Polícia Fabio Luiz Da Silva Souza
4,5,7,007ª DP - Santa Teresa,"Rua Francisco de Castro, 5 - Santa Teresa, Rio...",Delegado de Polícia Carlos Alberto Meirelles D...
...,...,...,...,...,...
132,133,159,159ª DP - Cachoeira de Macacu,"Avenida Lord Baden Powel, 93 - Centro, Cachoei...",Delegado de Polícia Clovis Souza Moreira
133,134,165,165ª DP - Mangaratiba,"Estrada São João Marcos, s/n - Praia do Saco, ...",Delegado de Polícia Anderson Ribeiro Pinto
134,135,166,166ª DP - Angra dos Reis,"Rua Doutor Coutinho, 6 - Centro, Angra dos Rei...",Delegado de Polícia Vilson de Almeida Silva
135,136,167,167ª DP - Paraty,"Rua Roberto da Silveira , s/n - Vila Colonial,...",Delegado de Polícia Marcelo dos Santos Haddad


# Fechando a conexão com o DW

In [36]:
# Confirmar a transação
conexaoDW.commit()

# Fechar a conexão
conexaoDW.close()


# Coletando dados dos Batalhões de Polícia Militar (BPM)

In [6]:
# Criar DataFrame com os dados dos BPMs
path_BPM = os.path.join(path, 'BPM.csv')
tbLogBPM = pd.read_csv(path_BPM)

# Criando coluna com a data da carga
tbLogBPM['dtCarga'] = dtCarga